# Perceptron Classification Tutorial / パーセプトロン分類チュートリアル


In this tutorial, you will learn:
このチュートリアルでは、以下を学習します：

- **Binary classification with perceptron** / パーセプトロンによる二値分類
- **Error correction learning algorithm** / 誤り訂正学習アルゴリズム  
- **Weight update visualization** / 重み更新の可視化
- **Cost function analysis** / コスト関数解析


The perceptron is one of the simplest neural network models for binary classification. It uses an error correction learning algorithm to find a linear decision boundary that separates two classes.

パーセプトロンは二値分類のための最もシンプルなニューラルネットワークモデルの一つです。誤り訂正学習アルゴリズムを使用して、2つのクラスを分離する線形決定境界を見つけます。

In [ ]:
# Import required libraries / 必要なライブラリをインポート
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.animation as animation

np.random.seed(12345)

sns.set()
sns.set_context('talk')
plt.rcParams['axes.grid'] = False
plt.rcParams['font.size'] = 20
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams['image.cmap'] = 'plasma'

print("Libraries imported successfully / ライブラリのインポートが完了しました")

## Q1: Generate 2D Gaussian Classification Data / 2次元ガウス分類データの生成

Generate two classes of 2D data points using multivariate normal distributions for binary classification.

二値分類のために多変量正規分布を使用して2つのクラスの2次元データポイントを生成します。

In [ ]:
# Generate 2D Gaussian data for binary classification / 二値分類用2次元ガウスデータ生成
mu1 = np.array([1, 3])  # Class 1 center / クラス1の中心
mu2 = np.array([3, 1])  # Class 2 center / クラス2の中心
cov = 2 * np.array([[1.0, 0.2], [0.2, 0.3]])  # Shared covariance matrix / 共通共分散行列

N1 = 50  # Number of samples for class 1 / クラス1のサンプル数
N2 = 30  # Number of samples for class 2 / クラス2のサンプル数

x1 = np.random.multivariate_normal(mu1, cov, N1)
x2 = np.random.multivariate_normal(mu2, cov, N2)
x = np.vstack((x1, x2))

y1 = np.ones(N1)
y2 = -np.ones(N2)
y = np.hstack((y1, y2))

plt.figure(figsize=(8, 6))
plt.plot(x[y==1, 0], x[y==1, 1], 'ro', alpha=0.7, label='Class +1', markersize=8)
plt.plot(x[y==-1, 0], x[y==-1, 1], 'bo', alpha=0.7, label='Class -1', markersize=8)
plt.grid(True, alpha=0.3)
plt.xlabel('X1')
plt.ylabel('X2')
plt.title('2D Binary Classification Data')
plt.legend()
plt.axis('equal')
plt.show()

print(f"Generated {N1} samples for class +1 and {N2} samples for class -1")
print(f"クラス+1に{N1}個、クラス-1に{N2}個のサンプルを生成しました")

## Q2: Implement Perceptron Prediction Function / パーセプトロン予測関数の実装

Implement the core functions for perceptron prediction and weight computation.

パーセプトロンの予測と重み計算のためのコア関数を実装します。

In [ ]:
def weight_sum(w, x):
    """
    Compute weighted sum for perceptron
    パーセプトロンの重み付き和を計算
    
    Parameters:
    w: weight vector (D+1 dimensions including bias) / 重みベクトル（バイアス含むD+1次元）
    x: input data matrix (N, D) / 入力データ行列 (N, D)
    
    Returns:
    Weighted sum for each input / 各入力の重み付き和
    """
    N = x.shape[0]
    x_with_bias = np.hstack((np.ones((N, 1)), x))
    return x_with_bias @ w

def predict(x, w):
    """
    Perceptron prediction function (returns ±1)
    パーセプトロン予測関数（±1を返す）
    
    Parameters:
    x: input data / 入力データ
    w: weight vector / 重みベクトル
    
    Returns:
    Predicted class labels (±1) / 予測クラスラベル（±1）
    """
    return np.sign(weight_sum(w, x))

def draw_decision_boundary(x, y, w, xlim=(-3, 12), ylim=(-6, 6)):
    """
    Draw decision boundary line
    決定境界線を描画
    
    Parameters:
    x, y: data points and labels / データポイントとラベル
    w: weight vector / 重みベクトル
    xlim, ylim: plot limits / プロット範囲
    """
    xx1 = np.linspace(xlim[0], xlim[1], 100)
    xx2 = -(w[0] + w[1] * xx1) / w[2]
    
    plt.plot(xx1, xx2, 'g-', linewidth=3, label='Decision Boundary')
    return plt.gca().lines[-1]

print("Perceptron functions implemented successfully")
print("パーセプトロン関数の実装が完了しました")

## Q3: Implement Error Correction Learning / 誤り訂正学習の実装

Implement the error correction learning algorithm to update perceptron weights.

パーセプトロンの重みを更新する誤り訂正学習アルゴリズムを実装します。

In [ ]:
def update_weights(w, x, t, eta=0.0025):
    """
    Update perceptron weights using error correction learning
    誤り訂正学習を使用してパーセプトロンの重みを更新
    
    Parameters:
    w: current weight vector / 現在の重みベクトル
    x: input data / 入力データ
    t: target labels / 目標ラベル
    eta: learning rate / 学習率
    
    Returns:
    Updated weight vector / 更新された重みベクトル
    """
    ws = weight_sum(w, x)
    
    N = x.shape[0]
    x_with_bias = np.hstack((np.ones((N, 1)), x))
    
    misclassified = t * ws < 0
    
    if np.any(misclassified):
        delta = eta * np.sum(t[misclassified, np.newaxis] * x_with_bias[misclassified, :], axis=0)
        w = w + delta
    
    return w

def compute_cost(w, x, t):
    """
    Compute perceptron cost function
    パーセプトロンのコスト関数を計算
    
    Parameters:
    w: weight vector / 重みベクトル
    x: input data / 入力データ
    t: target labels / 目標ラベル
    
    Returns:
    Cost function value / コスト関数値
    """
    ws = weight_sum(w, x)
    misclassified = t * ws < 0
    
    if np.any(misclassified):
        return -np.sum(ws[misclassified] * t[misclassified])
    else:
        return 0.0

print("Error correction learning functions implemented")
print("誤り訂正学習関数の実装が完了しました")

## Q4: Visualize Learning Process / 学習過程の可視化

Train the perceptron and visualize how the decision boundary evolves during learning.

パーセプトロンを訓練し、学習中に決定境界がどのように変化するかを可視化します。

In [ ]:
# Initialize random weights / ランダムな重みで初期化
w = np.random.uniform(low=-1, high=1, size=3)
print(f"Initial weights: {w}")
print(f"初期重み: {w}")

xlim = [-3, 12]
ylim = [-6, 6]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

ax1.plot(x[y==1, 0], x[y==1, 1], 'ro', alpha=0.7, label='Class +1', markersize=8)
ax1.plot(x[y==-1, 0], x[y==-1, 1], 'bo', alpha=0.7, label='Class -1', markersize=8)
ax1.set_xlim(xlim)
ax1.set_ylim(ylim)
ax1.grid(True, alpha=0.3)
ax1.set_xlabel('X1')
ax1.set_ylabel('X2')
ax1.set_title('Perceptron Learning Result')
ax1.legend()

costs = []
weights_history = []

initial_cost = compute_cost(w, x, y)
costs.append(initial_cost)
weights_history.append(w.copy())

print(f"Initial cost: {initial_cost:.4f}")
print(f"初期コスト: {initial_cost:.4f}")

max_epochs = 200
for epoch in range(max_epochs):
    w_old = w.copy()
    w = update_weights(w, x, y, eta=0.0025)
    cost = compute_cost(w, x, y)
    costs.append(cost)
    weights_history.append(w.copy())
    
    if (epoch + 1) % 50 == 0:
        print(f"Epoch {epoch+1}: Cost = {cost:.4f}")
        print(f"エポック {epoch+1}: コスト = {cost:.4f}")
    
    if cost == 0.0:
        print(f"Converged at epoch {epoch+1}")
        print(f"エポック {epoch+1} で収束しました")
        break

draw_decision_boundary(x, y, w, xlim, ylim)

print(f"Final weights: {w}")
print(f"最終重み: {w}")
print(f"Final cost: {costs[-1]:.4f}")
print(f"最終コスト: {costs[-1]:.4f}")

plt.tight_layout()
plt.show()

## Q5: Analyze Cost Function Evolution / コスト関数の変化解析

Analyze how the cost function decreases during the learning process.

学習過程でコスト関数がどのように減少するかを分析します。

In [ ]:
# Plot cost function evolution / コスト関数の変化をプロット
plt.figure(figsize=(10, 6))
plt.semilogy(costs, 'b-', linewidth=2, marker='o', markersize=4)
plt.grid(True, alpha=0.3)
plt.xlabel('Epoch')
plt.ylabel('Cost Function Value')
plt.title('Perceptron Learning: Cost Function Evolution')
plt.show()

predictions = predict(x, w)
accuracy = np.mean(predictions == y)
print(f"Final classification accuracy: {accuracy:.2%}")
print(f"最終分類精度: {accuracy:.2%}")

weights_array = np.array(weights_history)
plt.figure(figsize=(12, 4))

for i in range(3):
    plt.subplot(1, 3, i+1)
    plt.plot(weights_array[:, i], 'g-', linewidth=2)
    plt.grid(True, alpha=0.3)
    plt.xlabel('Epoch')
    plt.ylabel(f'Weight w{i}')
    plt.title(f'Weight w{i} Evolution')

plt.tight_layout()
plt.show()

print("\nPerceptron learning analysis completed!")
print("パーセプトロン学習解析が完了しました！")

## Summary / まとめ

In this tutorial, you learned:
このチュートリアルでは以下を学習しました：

1. **Data Generation**: How to create 2D Gaussian data for binary classification
   **データ生成**: 二値分類のための2次元ガウスデータの作成方法

2. **Perceptron Implementation**: Core functions for prediction and weight computation
   **パーセプトロン実装**: 予測と重み計算のためのコア関数

3. **Error Correction Learning**: Algorithm to update weights based on misclassified samples
   **誤り訂正学習**: 誤分類されたサンプルに基づく重み更新アルゴリズム

4. **Visualization**: How to visualize the learning process and decision boundary evolution
   **可視化**: 学習過程と決定境界の変化の可視化方法

5. **Analysis**: Cost function evolution and convergence behavior
   **解析**: コスト関数の変化と収束挙動

The perceptron is a fundamental building block for more complex neural networks. Understanding its learning mechanism provides the foundation for advanced machine learning techniques.

パーセプトロンは、より複雑なニューラルネットワークの基本的な構成要素です。その学習メカニズムを理解することで、高度な機械学習技術の基礎が得られます。